In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
fname = 'files/Cal5DoF05222023-post FDPR V3.xlsx'
sheetname = 'Input for Tyler analysis'
fname = 'files/5DoF inputs - BA4C6.xlsx'
#sheetname = 'Input for Tyler analysis'

adm_log_filepath = 'files/ADM Ops Log.xlsx'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     2.210756  -0.290287   0.814773 -0.067157  -0.008609
PR2   -108.29734 -47.312164  23.498813 -4.997343  10.642332
PR3   112.517982 -46.993301  23.395663 -4.968657 -10.638305
PR4   104.360724  58.166598  31.846759  5.198137 -10.361601
PR5  -100.146895  57.964945  31.766209  5.182439  10.360469 

Provided 5DOF encoder values: 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PR1       2.187817  -0.252234   0.799781 -0.066404  -0.006532
PR2    -108.202269 -47.366198  23.386051 -4.997393  10.641322
PR3     112.470420 -46.695257  23.496738 -4.967277 -10.638100
PR4     104.167634  58.392530  32.042145  5.202955 -10.359213
PR5    -100.204604  57.838185  31.759767  5.181824  10.361290


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [5]:
#p_null_PAT_baseline_encoder = pd.DataFrame(dict({'X':88.435, 'Y':109.266,'Z':0,
#                                                  'Rx':0.1018,'Ry':0.0856,
#                                                  'z_sMATF':4.027174*1000.,
#                                                  'z_PATB':3.455684*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':0.0041,
#                                                  'sMATF AC EL':0.006,
#                                                  'PATB AC AZ':0.0041,
#                                                  'PATB AC EL':0.0035,
#                                                  'sMATF Pri LED X':811.9,
#                                                  'sMATF Pri LED Y':604.6,
#                                                  'PATB Pri LED X':812.6,
#                                                  'PATB Pri LED Y':605.9,
#                                                  'date':'4/1/2023'
#                                                 }),index=['PATB'])
pat_target = 'B'
z_target = 'mirror'
optimize_PAT_for_imperfect_nulling = True
baseline_ADM_plateau_name = '6/1 BA4'
update_ADM_plateau_name = 'BA5 with null'
#baseline_ADM_plateau_name = 'baseline'
#update_ADM_plateau_name = 'post-perturb'
#output_filename = 'Poses - BA5C2 ADM update from BA4C6 - Evan (from nulled measurements).xlsx'
output_filename = '_temp poses.xlsx'


p_null_PAT_baseline_encoder = get_data_from_ADM_log(baseline_ADM_plateau_name,z_type=z_target,pat_target = pat_target,index_name='PAT'+pat_target,print_details=True,filepath=adm_log_filepath) 
p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)

if pat_target=='B':
    p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
elif pat_target=='A':
    p_null_offset = np.array([90.,-110.,-10.])
p_null_PAT_baseline_encoder


X                               88.936
Y                               107.73
Z                                  0.0
Rx                              0.0143
Ry                              0.0735
z_sMATF                       4021.479
z_PAT                         3472.218
z_type                          mirror
sMATF AC AZ                    -0.0018
sMATF AC EL                     0.0042
PAT AC AZ                      -0.0011
PAT AC EL                       0.0022
sMATF Pri LED X                 813.64
sMATF Pri LED Y                 597.54
PAT Pri LED X                   812.96
PAT Pri LED Y                    596.8
date               2023-06-01 00:00:00
Name: PATB, dtype: object


,X,Y,Z,Rx,Ry,z_sMATF,z_PAT,z_type,sMATF AC AZ,sMATF AC EL,PAT AC AZ,PAT AC EL,sMATF Pri LED X,sMATF Pri LED Y,PAT Pri LED X,PAT Pri LED Y,date
PATB,88.936,107.73,0.0,0.0143,0.0735,4021.479,3472.218,mirror,-0.0018,0.0042,-0.0011,0.0022,813.64,597.54,812.96,596.8,2023-06-01


### 1.1.2 Update the PAT pose to account for imperfect nulling.

In [6]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_baseline_encoder_original = p_null_PAT_baseline_encoder.copy()
if optimize_PAT_for_imperfect_nulling == True:
    print(p_null_PAT_baseline_encoder.squeeze(),'\n')
    p_null_PAT_baseline_encoder = optimize_p_null_PAT_encoders(p_null_PAT_baseline_encoder,p_null_offset)
    p_null_PAT_baseline = calculate_5DOF_from_encoders(p_null_PAT_baseline_encoder)
    print('\n',p_null_PAT_baseline_encoder.squeeze())

X                               88.936
Y                               107.73
Z                                  0.0
Rx                              0.0143
Ry                              0.0735
z_sMATF                       4021.479
z_PAT                         3472.218
z_type                          mirror
sMATF AC AZ                    -0.0018
sMATF AC EL                     0.0042
PAT AC AZ                      -0.0011
PAT AC EL                       0.0022
sMATF Pri LED X                 813.64
sMATF Pri LED Y                 597.54
PAT Pri LED X                   812.96
PAT Pri LED Y                    596.8
date               2023-06-01 00:00:00
Name: PATB, dtype: object 

Starting GSARX/RY:  0.0143 0.0735
Optimal GSARX/RY:  0.0148559258 0.075544739
Starting HTSA/VTSA:  88.936 107.73
Optimal HTSA/VTSA:  88.9403482 107.6817216

 X                            88.940348
Y                           107.681722
Z                                  0.0
Rx                            0.01

#### 1.1.5 Form a dataframe with the values supplied so far

In [7]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_sMATF'] - \
    p_null_PAT_baseline_encoder.loc['PAT'+pat_target,'z_PAT']
df = pd.concat([df,p_null_PAT_baseline])
print(p_null_PAT_baseline)
df.loc['PAT'+pat_target,'color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

              X          Y         Z        Rx        Ry
PATB  89.264361  107.91945 -0.677895  0.014127  0.073466


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,2.210756,-0.290287,0.814773,-0.067157,-0.008609,crimson,-0.194154,250.325498,1267.643071
PR2,-108.29734,-47.312164,23.498813,-4.997343,10.642332,crimson,238.626077,352.179123,1220.085779
PR3,112.517982,-46.993301,23.395663,-4.968657,-10.638305,crimson,-238.536825,351.572863,1220.278063
PR4,104.360724,58.166598,31.846759,5.198137,-10.361601,crimson,-232.401148,130.772934,1264.306131
PR5,-100.146895,57.964945,31.766209,5.182439,10.360469,crimson,232.376025,131.119798,1264.274823
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN
PATB,89.264361,107.91945,-0.677895,0.014127,0.073466,darkblue,1.656797,248.526688,1267.995894
sMPA,2.04,248.465,1268.826,NaN,NaN,purple,0.000129,0.411468,0.911424


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [8]:
start_coords = np.array(df.loc['PAT'+pat_target,['X','Y','Z']])

rx = df.loc['PAT'+pat_target,'Rx']
ry = df.loc['PAT'+pat_target,'Ry']
print(rx,ry)
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PAT'+pat_target,['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

0.014126519109576931 0.07346630221253209


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,2.210756,-0.290287,0.814773,-0.067157,-0.008609,crimson,-0.194154,250.325498,1267.643071
PR2,-108.29734,-47.312164,23.498813,-4.997343,10.642332,crimson,238.626077,352.179123,1220.085779
PR3,112.517982,-46.993301,23.395663,-4.968657,-10.638305,crimson,-238.536825,351.572863,1220.278063
PR4,104.360724,58.166598,31.846759,5.198137,-10.361601,crimson,-232.401148,130.772934,1264.306131
PR5,-100.146895,57.964945,31.766209,5.182439,10.360469,crimson,232.376025,131.119798,1264.274823
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN
PATB,89.264361,107.91945,-0.677895,0.014127,0.073466,darkblue,1.656797,248.526688,1267.995894
sMPA,2.04,248.465,1268.826,NaN,NaN,purple,0.000129,0.411468,0.911424
sMATF,-0.044106,103.717296,549.786526,0.014127,0.073466,orange,0.704281,105.645167,539.007054


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [9]:
#p_null_PAT_update_encoder = pd.DataFrame(dict({'X':88.526, 'Y':108.364,'Z':0,
#                                                  'Rx':0.1287,'Ry':0.0788,
#                                                  'z_sMATF':4.031636*1000.,
#                                                  'z_PATB':3.458517*1000.,
#                                                  'z_type':'retro',
#                                                  'sMATF AC AZ':-0.0022,
#                                                  'sMATF AC EL':0.0042,
#                                                  'PATB AC AZ':-0.0008,
#                                                  'PATB AC EL':0.0001,
#                                                  'sMATF Pri LED X':811.7,
#                                                  'sMATF Pri LED Y':597.6,
#                                                  'PATB Pri LED X':812.1,
#                                                  'PATB Pri LED Y':598.8,
#                                                  'date':'4/3/2023'
#                                                 }),index=['PATB_update'])
p_null_PAT_update_encoder = get_data_from_ADM_log(update_ADM_plateau_name,z_type=z_target,pat_target = pat_target,index_name='PAT'+pat_target+'_update',print_details=True,filepath=adm_log_filepath) 
p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)
p_null_PAT_update

X                               88.893
Y                              108.635
Z                               -0.004
Rx                              0.1078
Ry                              0.0877
z_sMATF                       4021.926
z_PAT                         3472.155
z_type                          mirror
sMATF AC AZ                    -0.0004
sMATF AC EL                     0.0022
PAT AC AZ                      -0.0008
PAT AC EL                       0.0032
sMATF Pri LED X                 810.66
sMATF Pri LED Y                 601.65
PAT Pri LED X                   810.69
PAT Pri LED Y                   601.72
date               2023-06-01 00:00:00
Name: PATB_update, dtype: object


,X,Y,Z,Rx,Ry
PATB_update,89.219107,108.874422,-0.686118,0.107094,0.085616


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [10]:
#Apply a correction to account for imperfect nulling between PATB/sMATF and overwrite the values in the p_null_PAT_baseline_encoder DataFrame with the optimal ones.

#Save a copy of the p_null_PATB dataframe, which we'll write to an Excel file later
p_null_PAT_update_encoder_original = p_null_PAT_update_encoder.copy()

if optimize_PAT_for_imperfect_nulling == True:
    p_null_PAT_update_encoder = optimize_p_null_PAT_encoders(p_null_PAT_update_encoder,p_null_offset)
    p_null_PAT_update = calculate_5DOF_from_encoders(p_null_PAT_update_encoder)

Starting GSARX/RY:  0.1078 0.0877
Optimal GSARX/RY:  0.10756324340000001 0.08664931000000001
Starting HTSA/VTSA:  88.893 108.635
Optimal HTSA/VTSA:  88.8939112 108.63857250000001


#### 1.2.2 Use the newly-provided data to calculate an updated position for PATB and sMATF

In [12]:
dz_update = p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_sMATF'] - \
    p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update','z_PAT'] + \
    (p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update',['Y']]-p_null_PAT_baseline_encoder.loc['PAT'+pat_target,['Y']]).values[0]*np.sin(np.deg2rad(GSA_angle_WCS_deg+p_null_PAT_update_encoder.loc['PAT'+pat_target+'_update',['Rx']].astype(float).values[0]))
df = pd.concat([df,p_null_PAT_update])
df.loc['PAT'+pat_target+'_update','color'] = 'purple'
update_uvec(df,'PAT'+pat_target+'_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PAT'+pat_target+'_update',['X','Y','Z']])

rx = df.loc['PAT'+pat_target+'_update','Rx']
ry = df.loc['PAT'+pat_target+'_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PAT'+pat_target+'_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,2.210756,-0.290287,0.814773,-0.067157,-0.008609,crimson,-0.194154,250.325498,1267.643071
PR2,-108.29734,-47.312164,23.498813,-4.997343,10.642332,crimson,238.626077,352.179123,1220.085779
PR3,112.517982,-46.993301,23.395663,-4.968657,-10.638305,crimson,-238.536825,351.572863,1220.278063
PR4,104.360724,58.166598,31.846759,5.198137,-10.361601,crimson,-232.401148,130.772934,1264.306131
PR5,-100.146895,57.964945,31.766209,5.182439,10.360469,crimson,232.376025,131.119798,1264.274823
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN
PATB,89.264361,107.91945,-0.677895,0.014127,0.073466,darkblue,1.656797,248.526688,1267.995894
sMPA,2.04,248.465,1268.826,NaN,NaN,purple,0.000129,0.411468,0.911424
sMATF,-0.044106,103.717296,549.786526,0.014127,0.073466,orange,0.704281,105.645167,539.007054
PATB_update,89.22003,108.877999,-0.686139,0.106858,0.084566,purple,1.907104,246.474095,1268.396125


In [13]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PAT'+pat_target+'->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  0.32415040420858077  mm


## 1.3 Calculate sMATF and PAT deltas

In [14]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
#sMATF_delta.loc['Z'] += 
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PAT'+pat_target+'_update',['X','Y','Z','Rx','Ry']] - df.loc['PAT'+pat_target,['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PAT'+pat_target+' (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')

sMATF (X,Y,Z) deltas: 
 X    0.060636
Y     0.13278
Z    0.319406
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [16]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X       0.01653
Y    103.850076
Z    550.105931
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [17]:
update_rot1 = R.from_euler('XY',[df.loc['PAT'+pat_target,'Rx']+GSA_angle_WCS_deg,df.loc['PAT'+pat_target,'Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [18]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PR1,2.163165,0.731032,0.966148,0.025574,0.002490,crimson,0.056155,248.273543,1268.046565
PR2,-108.342474,-46.323188,23.595026,-4.904773,10.653399,crimson,238.871363,350.194693,1220.608863
PR3,112.472837,-46.012767,23.450489,-4.875768,-10.627238,crimson,-238.291518,349.606727,1220.890714
PR4,104.321314,59.133634,32.073318,5.290663,-10.350558,crimson,-232.156166,128.735590,1264.560204
PR5,-100.186324,58.940085,32.03125,5.275375,10.371512,crimson,232.621007,129.064383,1264.441260
sMask,2.06339,115.302407,588.617062,NaN,NaN,g,NaN,NaN,NaN
PATB,89.220737,108.939649,-0.367905,0.106858,0.084565,darkblue,1.907102,246.474093,1268.396126
sMPA,2.242726,247.43377,1269.378323,NaN,NaN,purple,0.000318,0.409993,0.912089
sMATF,0.01653,103.850076,550.105931,0.106858,0.084565,orange,0.810682,104.772638,539.177187
PATB_update,89.176442,109.898212,-0.374589,0.199589,0.095664,purple,2.157405,244.420844,1268.792987


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [19]:
df_update_encoder_space = convert_df_to_encoder_space(df_update)
print('Poses calculated using PAT'+pat_target+' and the PAT '+z_target)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

Poses calculated using PATB and the PAT mirror
Updated Poses (in encoder space): 
               X          Y          Z        Rx         Ry
PR1    2.138863   0.770344   0.950754  0.026304   0.004572
PR2 -108.249306 -46.371321  23.478986 -4.904729  10.652422
PR3  112.423147 -45.708420  23.548210 -4.874262 -10.626989
PR4  104.127459  59.355654  32.271674  5.295555 -10.348123
PR5 -100.245410  58.811642  32.026732  5.274800  10.372370 


Baseline Poses (in encoder space), just for easy reference: 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PR1       2.187817  -0.252234   0.799781 -0.066404  -0.006532
PR2    -108.202269 -47.366198  23.386051 -4.997393  10.641322
PR3     112.470420 -46.695257  23.496738 -4.967277 -10.638100
PR4     104.167634  58.392530  32.042145  5.202955 -10.359213
PR5    -100.204604  57.838185  31.759767  5.181824  10.361290


#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [20]:
print_extra_diagnostics = False
poses_to_display = [val for val in df.index if 'PDI' in val or 'PR' in val]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update),'\n\n')
    print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update),'\n\n')
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df),'\n\n')  
    

# 2 Write the new poses to an Excel file in the current directory

In [21]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='ADM',baseline_filepath=fname,
                             columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             baseline_ADM_plateau_name=baseline_ADM_plateau_name,update_ADM_plateau_name=update_ADM_plateau_name,
                             df=df,df_encoders=pose_encoders,df_update=df_update,df_update_encoders=df_update_encoder_space,
                            focal_length=focal_length,p_null_PAT_baseline_encoder_original=p_null_PAT_baseline_encoder_original,
                            p_null_PAT_update_encoder_original=p_null_PAT_update_encoder_original)

**Writing to Excel complete.**
**Filename:  _temp poses.xlsx
